In [1]:
using StatisticalRethinking, Distributed, JLD
using Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
line[:x] = convert(Array{Float64,1}, df2[:weight]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

# Model Specification
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)

# Initial Values
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

# Tuning Parameters
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

# Define sampling scheme

scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

# MCMC Simulation

sim = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:12:05 of 0:12:06 remaining]
Chain 1:  10% [0:01:03 of 0:01:10 remaining]
Chain 1:  20% [0:00:58 of 0:01:12 remaining]
Chain 1:  30% [0:00:52 of 0:01:14 remaining]
Chain 1:  40% [0:00:44 of 0:01:13 remaining]
Chain 1:  50% [0:00:37 of 0:01:14 remaining]
Chain 1:  60% [0:00:30 of 0:01:14 remaining]
Chain 1:  70% [0:00:22 of 0:01:15 remaining]
Chain 1:  80% [0:00:15 of 0:01:15 remaining]
Chain 1:  90% [0:00:08 of 0:01:16 remaining]
Chain 1: 100% [0:00:00 of 0:01:16 remaining]

Chain 2:   0% [0:00:02 of 0:00:02 remaining]
Chain 2:  10% [0:00:43 of 0:00:48 remaining]
Chain 2:  20% [0:00:51 of 0:01:04 remaining]
Chain 2:  30% [0:00:48 of 0:01:09 remaining]
Chain 2:  40% [0:00:44 of 0:01:13 remaining]
Chain 2:  50% [0:00:38 of 0:01:16 remaining]
Chain 2:  60% [0:00:31 of 0:01:17 remaining]
Chain 2:  70% [0:00:23 of 0:01:18 remaining]
Chain 2:  80% [0:00:16 of 0:01:18 remaining]
Chain 2:  90% [0:00:08 of 0:01:19 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[29.8865 113.71 0.905738; 26.6634 113.995 0.900735; … ; 23.9113 115.238 0.87488; 25.9669 115.649 0.868332]

[26.2989 113.783 0.902103; 26.515 114.601 0.895489; … ; 24.4435 113.943 0.911527; 25.4821 113.941 0.905677]

[23.2469 113.789 0.904543; 21.3024 114.927 0.884294; … ; 26.6104 113.885 0.91133; 25.2642 115.094 0.882973]

Show draws summary

In [3]:
describe(sim)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean          SD       Naive SE       MCSE        ESS   
     s2  26.14968609 1.987091681 0.0120930549 0.0197241434 9000.0000
beta[1] 113.91139777 1.913163198 0.0116431404 0.0378133006 2559.8520
beta[2]   0.90435757 0.042043533 0.0002558688 0.0008229053 2610.3499

Quantiles:
            2.5%       25.0%       50.0%        75.0%       97.5%   
     s2  22.5461433  24.7761233  26.0584019  27.40497062  30.3374232
beta[1] 110.2483969 112.5938568 113.8774339 115.17813382 117.7088036
beta[2]   0.8208175   0.8764367   0.9051815   0.93292284   0.9835131



End of `clip_38.0m.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*